 <center><h1> Segmented and Clustered the Neighborhoods Project

## 1. Import Library

In [62]:
#import nescessary library
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim

import folium

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

import matplotlib.pyplot as plt


print('The library was import')

The library was import


## 2. Collect data and Import to Notebook

### 2.1 Manipulate data from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

url = requests.get(url).text

In [3]:
data_toronto_url = BeautifulSoup(url, 'html.parser')

In [4]:
data_toronto_table_url = data_toronto_url.find('table')

In [5]:
df_toronto_table_emply = []

for row in data_toronto_table_url.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        df_toronto_table_emply.append(cell)

In [6]:
df_toronto_table_emply

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'PostalCode': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'PostalCode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'PostalCode': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'PostalCode': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'PostalCode': 'M6B', 'Borough': 'North York', 'N

In [7]:
df_toronto = pd.DataFrame(df_toronto_table_emply)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df_toronto['Borough']=df_toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
# see the shape of data
df_toronto.shape

(103, 3)

### 2.2 Using data relative location provided by Coursera course

In [11]:
# read csv canada file consist of three columns: PostalCode, Borough, and Neighborhood (file complete cleaning on assignment 1)
df_toronto_location = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [12]:
df_toronto_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_toronto_location.shape

(103, 3)

In [14]:
df_toronto.shape

(103, 3)

### 2.3 Merged two dataframe ( one collected in Wipipedid and one provided by Coursera)

In [15]:
#Using merged function in connect two dataframe have key_id is postal_code 
merged_df_data_toronto = pd.merge(left= df_toronto, right= df_toronto_location, 
                                  left_on= 'PostalCode', 
                                  right_on= "Postal Code")

In [16]:
merged_df_data_toronto.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [17]:
merged_df_data_toronto =merged_df_data_toronto.drop(['Postal Code'], axis= "columns")

In [18]:
merged_df_data_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [19]:
df_toronto_official = merged_df_data_toronto

In [20]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'
      .format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## 3. Visualize map relative dataset

In [21]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
map_toronto

In [22]:
# add markers to map
for lat, lng, label in zip(df_toronto_official['Latitude'], df_toronto_official['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
df_toronto['Borough'].value_counts()

North York                24
Downtown Toronto          17
Scarborough               17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East York                  4
East Toronto               4
Queen's Park               1
East York/East Toronto     1
Mississauga                1
Downtown Toronto Stn A     1
Etobicoke Northwest        1
East Toronto Business      1
Name: Borough, dtype: int64

## 4. Collect data relative venues on Fousquare 

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
CLIENT_ID = 'FSEB55GSJNOFADCDLXWPRQH4OJW2RWTBEGZF4INFJ4BNVYLH' # My Foursquare ID
CLIENT_SECRET = 'JOVA5LKXAFYOEYWCXEU1IX2QNT2IJIJX0IYNGZ1KF31GDO2G' # My Foursquare Secret
ACCESS_TOKEN = 'EWDRDJCFCIXM3X2RWXK0JF1HH3ETL4KDL5LQ5BPHTFINGRYP' # My FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
code = 'BE5L1KYLVGNN1OJECTJGULAQSLY0RQA2A55O2WYUAEW5DRDM#_=_'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Code:'+ code)
print('Access_token:'+ ACCESS_TOKEN)

Your credentails:
CLIENT_ID: FSEB55GSJNOFADCDLXWPRQH4OJW2RWTBEGZF4INFJ4BNVYLH
CLIENT_SECRET:JOVA5LKXAFYOEYWCXEU1IX2QNT2IJIJX0IYNGZ1KF31GDO2G
Code:BE5L1KYLVGNN1OJECTJGULAQSLY0RQA2A55O2WYUAEW5DRDM#_=_
Access_token:EWDRDJCFCIXM3X2RWXK0JF1HH3ETL4KDL5LQ5BPHTFINGRYP


In [26]:
toronto_venues = getNearbyVenues(names=df_toronto_official['Neighborhood'],
                                   latitudes=df_toronto_official['Latitude'],
                                   longitudes=df_toronto_official['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(1332, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


## 5. Evaluate the top 5 venues for each of neighborhood in Toronto

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+str(hood)+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1     Coffee Shop  0.17
2             Pub  0.17
3  Sandwich Place  0.17
4             Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                   Pharmacy  0.05
3                Pizza Place  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3   Chinese Restaurant  0.25
4          Music Venue  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08

##  6. Analyze Each Neighborhood

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Yoga Studio,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pub,Sandwich Place,Gym,Mediterranean Restaurant,Men's Store,Metro Station,Movie Theater,Mexican Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Pizza Place,Middle Eastern Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Supermarket,Sushi Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Café,Chinese Restaurant,Music Venue,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Pub,Comfort Food Restaurant,Restaurant,Café,Butcher


# 7. Cluster neighboorhoods

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=None).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe

kmeans.labels_[0:10] 

<ipython-input-34-4dfc979619b4>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2])

In [35]:
neighborhoods_venues_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Neighborhood            100 non-null    object
 1   1st Most Common Venue   100 non-null    object
 2   2nd Most Common Venue   100 non-null    object
 3   3rd Most Common Venue   100 non-null    object
 4   4th Most Common Venue   100 non-null    object
 5   5th Most Common Venue   100 non-null    object
 6   6th Most Common Venue   100 non-null    object
 7   7th Most Common Venue   100 non-null    object
 8   8th Most Common Venue   100 non-null    object
 9   9th Most Common Venue   100 non-null    object
 10  10th Most Common Venue  100 non-null    object
dtypes: object(11)
memory usage: 8.7+ KB


In [36]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Yoga Studio,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pub,Sandwich Place,Gym,Mediterranean Restaurant,Men's Store,Metro Station,Movie Theater,Mexican Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Pizza Place,Middle Eastern Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Supermarket,Sushi Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Café,Chinese Restaurant,Music Venue,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Breakfast Spot,Pharmacy,Pub,Comfort Food Restaurant,Restaurant,Café,Butcher


In [37]:

#neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

neighborhoods_venues_sorted = neighborhoods_venues_sorted.astype(str)
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,1,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Monument / Landmark,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
1,M4A,North York,Victoria Village,1,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Yoga Studio,Miscellaneous Shop,Motel,Monument / Landmark,Modern European Restaurant,Mobile Phone Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Park,Bakery,Theater,Pub,Yoga Studio,Shoe Store,Breakfast Spot,Mexican Restaurant,Farmers Market
3,M6A,North York,"Lawrence Manor, Lawrence Heights",1,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Event Space,Coffee Shop,Miscellaneous Shop,Monument / Landmark,Modern European Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,1,Coffee Shop,Sushi Restaurant,Café,Diner,College Auditorium,Beer Bar,Smoothie Shop,Burger Joint,Burrito Place,Sandwich Place


In [46]:
toronto_merged = toronto_merged.dropna()

In [53]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [54]:
toronto_merged.shape

(100, 14)

In [55]:
df_data_toronto_map = pd.merge(left = toronto_merged , right= df_toronto_location, 
                                  left_on= 'PostalCode', 
                                  right_on= "Postal Code")
df_data_toronto_map = df_data_toronto_map.drop("Postal Code", axis = "columns")

In [56]:
#df_data_toronto_map["Cluster Labels"] = df_data_toronto_map["Cluster Labels"].astype(int)

In [57]:
df_data_toronto_map.head()

,PostalCode,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,M3A,North York,Parkwoods,1,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Monument / Landmark,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,43.753259,-79.329656
1,M4A,North York,Victoria Village,1,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Yoga Studio,Miscellaneous Shop,Motel,Monument / Landmark,Modern European Restaurant,Mobile Phone Shop,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Park,Bakery,Theater,Pub,Yoga Studio,Shoe Store,Breakfast Spot,Mexican Restaurant,Farmers Market,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",1,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Event Space,Coffee Shop,Miscellaneous Shop,Monument / Landmark,Modern European Restaurant,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,1,Coffee Shop,Sushi Restaurant,Café,Diner,College Auditorium,Beer Bar,Smoothie Shop,Burger Joint,Burrito Place,Sandwich Place,43.662301,-79.389494


In [58]:
df_data_toronto_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PostalCode              100 non-null    object 
 1   Borough                 100 non-null    object 
 2   Neighborhood            100 non-null    object 
 3   Cluster Labels          100 non-null    int32  
 4   1st Most Common Venue   100 non-null    object 
 5   2nd Most Common Venue   100 non-null    object 
 6   3rd Most Common Venue   100 non-null    object 
 7   4th Most Common Venue   100 non-null    object 
 8   5th Most Common Venue   100 non-null    object 
 9   6th Most Common Venue   100 non-null    object 
 10  7th Most Common Venue   100 non-null    object 
 11  8th Most Common Venue   100 non-null    object 
 12  9th Most Common Venue   100 non-null    object 
 13  10th Most Common Venue  100 non-null    object 
 14  Latitude                100 non-null    flo

In [59]:
df_data_toronto_map.to_csv('C:/Users/hp/Documents/Jupiter_Notebook/Applied Data Science Capstone/Assignment_Toronto_week 3/Assignment_Toronto/Data_Toronto_complete.csv')

## 8. Visualize finally results 

In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_data_toronto_map['Latitude'], 
                                  df_data_toronto_map['Longitude'], 
                                  df_data_toronto_map['Neighborhood'], 
                                  df_data_toronto_map['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
         [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

# THE END